In [1]:
# библиотеки
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

2024-06-23 02:45:17.668681: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 02:45:17.720800: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 02:45:17.952312: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 02:45:18.996703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# загрузка датасета
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [3]:
# предподготовка данных
# нормируем и приведем к требуемому формату данные
x_train_conv = x_train.astype('float32') / 255.
x_test_conv  = x_test.astype('float32') / 255.
x_train_conv = np.reshape(x_train_conv, (len(x_train), 28, 28, 1))
x_test_conv  = np.reshape(x_test_conv,  (len(x_test),  28, 28, 1))
print('x_train_conv.shape =', x_train_conv.shape)
print('x_test_conv.shape  =', x_test_conv.shape)

# подготовка правильных ответов
y_train_OHE = to_categorical(y_train, 10)
y_test_OHE = to_categorical(y_test, 10)
print('y_train_OHE.shape =', y_train_OHE.shape)
print('y_test_OHE.shape  =', y_test_OHE.shape)

x_train_conv.shape = (60000, 28, 28, 1)
x_test_conv.shape  = (10000, 28, 28, 1)
y_train_OHE.shape = (60000, 10)
y_test_OHE.shape  = (10000, 10)


In [4]:
# нейронная сеть
x_inp = Input((28,28,1), name='Input')

x = Conv2D(4, (2, 2), padding='same', activation='relu', name='Conv_1_blok_1')(x_inp)
x = Conv2D(4, (2,2), padding='same', activation='relu', name='Conv_2_blok_1')(x)
x = MaxPooling2D((2,2), name='MaxPoling_blok_1')(x)

x = Conv2D(8, (2,2), padding='same', activation='relu', name='Conv_1_blok_2')(x)
x = Conv2D(8, (2,2), padding='same', activation='relu', name='Conv_2_blok_2')(x)
x = MaxPooling2D((2,2), name='MaxPoling_blok_2')(x)

x = Flatten(name='Flatten')(x)

x = Dense(10, activation='sigmoid', name='outer_Dense')(x)
model_conv = Model(x_inp, x)

In [5]:
# компиляция
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# callbacks
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
mch = ModelCheckpoint(
    "classifire.keras",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch'
)

In [9]:
# обучение
history = model_conv.fit(
    x = x_train_conv,
    y = y_train_OHE,
    batch_size = 128,
    epochs = 100,
    validation_data = (x_test_conv, y_test_OHE),
    verbose = 1,
    callbacks = [mch, es]
)

Epoch 1/100
465/469 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8827 - loss: 0.3748
Epoch 1: val_loss improved from inf to 0.17028, saving model to classifire.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8830 - loss: 0.3738 - val_accuracy: 0.9473 - val_loss: 0.1703
Epoch 2/100
466/469 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9479 - loss: 0.1653
Epoch 2: val_loss improved from 0.17028 to 0.12412, saving model to classifire.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9479 - loss: 0.1652 - val_accuracy: 0.9605 - val_loss: 0.1241
Epoch 3/100
467/469 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9609 - loss: 0.1222
Epoch 3: val_loss improved from 0.12412 to 0.10226, saving model to classifire.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9609 - loss: 0.1222 - val_accuracy: 0.9705 - val_loss: 0.1023
Epoch 4/100
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9693 - loss: 0.1010
Epoch 4: val_loss improved from 0.10226 to 

In [10]:
# копируем полученную модель на s3-хранилище
!aws --endpoint-url=https://storage.yandexcloud.net/ s3 cp classifire.keras s3://cherepanov-bucket/classifire.keras

upload: ./classifire.keras to s3://cherepanov-bucket/classifire.keras
